In [1]:
from netCDF4 import Dataset
from wrf import (to_np, getvar, smooth2d, get_cartopy, cartopy_xlim,
                 cartopy_ylim, latlon_coords, ALL_TIMES,interpline, CoordPair, xy_to_ll, ll_to_xy)
import pandas as pd
import numpy as np
import os
import shutil
import re
import netCDF4 as nc
import xarray as xr

In [2]:
#mask out seas

ds_lu = xr.open_dataset('wrfout_d04_2011-06-23_11:00:00')
lu = ds_lu['LU_INDEX']

lu = lu[0,:,:]

del lu.coords['XTIME']

lu_df = lu.to_dataframe().reset_index()

In [3]:
#load files-1620
ds1_1120 = xr.open_dataset('ssp245_t2_1120_era5_corrected_dailymax.nc')
t2_1120_dailymax = ds1_1120['ssp245_t2_1120_era5_corrected_dailymax']

In [4]:
t2_1120_dailymax_max = t2_1120_dailymax.max(dim='Time')

In [5]:
t2_1120_dailymax_max_df = t2_1120_dailymax_max.to_dataframe().reset_index()

In [6]:
t2_1120_dailymax_max_df['Time'] = '2016-06-01'

In [7]:
t2_1120_dailymax_max_df['Time'] = pd.to_datetime(t2_1120_dailymax_max_df['Time'])

In [8]:
t2_1120_dailymax_max_df = t2_1120_dailymax_max_df.rename(columns={"ssp245_t2_1120_era5_corrected_dailymax": "daily_max"})

# 9099 return years calculation

In [9]:
#load files-9099
ds1_9099 = xr.open_dataset('ssp585_t2_9099_era5_corrected_dailymax.nc')
t2_9099_dailymax = ds1_9099['ssp585_t2_9099_era5_corrected_dailymax']

In [10]:
t2_9099_dailymax_df = t2_9099_dailymax.to_dataframe().reset_index()

In [11]:
t2_9099_dailymax_df = t2_9099_dailymax_df.rename(columns={"ssp585_t2_9099_era5_corrected_dailymax": "daily_max"})

In [12]:
t2_9099_dailymax_df = pd.concat([t2_9099_dailymax_df,t2_1120_dailymax_max_df]).reset_index(drop=True)

In [14]:
t2_9099_latlon = t2_9099_dailymax_df.groupby(['south_north','west_east'])

In [15]:
t2_9099_dailymax_df['auxi_col'] = t2_9099_latlon['daily_max'].rank(ascending=False)

In [16]:
t2_9099_dailymax_df = t2_9099_dailymax_df.sort_values(['south_north','west_east','daily_max'], ascending = (True,True,False)).reset_index(drop=True)

In [17]:
t2_9099_dailymax_position = t2_9099_dailymax_df[t2_9099_dailymax_df['Time']=='2016-06-01'].reset_index(drop=True)

In [18]:
t2_9099_dailymax_position['reoccur_day'] = (10*365+3)/t2_9099_dailymax_position['auxi_col']

In [19]:
t2_9099_dailymax_position['reoccur_year'] = t2_9099_dailymax_position['reoccur_day']/365

In [21]:
t2_9099_dailymax_position = pd.merge(t2_9099_dailymax_position,lu_df,on=['south_north','west_east','XLAT','XLONG'],how='outer')

t2_9099_dailymax_position.loc[t2_9099_dailymax_position.LU_INDEX_x == 16, 'reoccur_year'] = 9999

In [22]:
#to csv
t2_9099_dailymax_position.to_csv('ssp585_t2_9099_ry_land.csv',index=False,columns=['south_north','west_east','XLAT', 'XLONG','reoccur_year'])